In [ ]:
from rtlsdr import RtlSdr
import pylab
import plotly.graph_objects as go

import numpy as np
from scipy.io import wavfile
from scipy.signal import butter, sosfilt
from scipy.signal import hilbert

from util.plotting import compute_fft_plot, compute_fft_plot_from_sample_rate, compute_frequency_response
from util.filtering import compute_lpf_coeff, low_pass_filter_complex_signal, low_pass_filter_real_signal, filter_complex_signal, compute_bandpass_coeff
from util.data_io import read_sdriq_data
from util.demodulation import demodulate_signal, chunked_demodulate_signal

In [ ]:
fs = 2048000
center_frequency = 98500000

signal = read_sdriq_data("/home/dominic/radio/data/long_test.sdriq")
signal = signal[:fs*5]
len(signal) / fs

In [ ]:
filtered_signal = low_pass_filter_complex_signal(signal=signal, cutoff_frequency=75E3, sample_rate=fs)
filtered_fs = fs / 10
filtered_signal = filtered_signal[::10]

In [ ]:
pylab.figure(figsize=(5,5))
freq, mag = compute_fft_plot_from_sample_rate(filtered_signal, sampling_rate=filtered_fs, center_frequency=0)
pylab.plot(freq, mag)
pylab.show()

In [ ]:
angle_diff = np.angle(
    np.conjugate(filtered_signal[:-1]) * filtered_signal[1:]
)
envelope = np.abs(hilbert(angle_diff))

In [ ]:
pylab.figure(figsize=(5,5))
freq, mag = compute_fft_plot_from_sample_rate(angle_diff, sampling_rate=filtered_fs, center_frequency=0)
pylab.plot(freq, mag)
pylab.show()

In [ ]:
pylab.figure(figsize=(5,5))
freq, mag = compute_fft_plot_from_sample_rate(envelope, sampling_rate=filtered_fs, center_frequency=0)
pylab.plot(freq, mag)
pylab.show()

In [ ]:
filtered_angle_diff = low_pass_filter_real_signal(signal=angle_diff, cutoff_frequency=15e3, sample_rate=filtered_fs)
filtered_envelope = low_pass_filter_real_signal(signal=envelope, cutoff_frequency=15e3, sample_rate=filtered_fs)


filter_angle_diff_downsampled = filtered_angle_diff[::5]
filtered_envelope_downsampled = filtered_envelope[::5]
output_fs = filtered_fs / 5

In [ ]:
pylab.figure(figsize=(5,5))
freq, mag = compute_fft_plot_from_sample_rate(filter_angle_diff_downsampled, sampling_rate=output_fs)
pylab.plot(freq, mag)
pylab.show()

In [ ]:
pylab.figure(figsize=(5,5))
freq, mag = compute_fft_plot_from_sample_rate(filtered_envelope_downsampled, sampling_rate=output_fs)
pylab.plot(freq, mag)
pylab.show()

In [ ]:
wavfile.write("data/test_angle_diff.wav", int(output_fs), filter_angle_diff_downsampled)
wavfile.write("data/test_envelope.wav", int(output_fs), filtered_envelope_downsampled)

In [ ]:
sos = compute_bandpass_coeff(low_cutoff=18.5E3, high_cutoff=19.5E3, sample_rate=filtered_fs)
freq, mag = compute_frequency_response(sos, filtered_fs)

fig = go.Figure()
fig.add_scatter(x=freq,y=mag)
fig.show()


In [ ]:
band_passed_diff = filter_complex_signal(angle_diff, sos)

pylab.figure(figsize=(5,5))
freq, mag = compute_fft_plot_from_sample_rate(band_passed_diff, sampling_rate=filtered_fs)
pylab.plot(freq, mag)
pylab.show()

In [ ]:
pylab.plot(band_passed_diff[:int(0.0075*filtered_fs)])

In [ ]:
import plotly.graph_objects as go

In [ ]:
fig = go.Figure()
fig.add_scatter(x=freq,y=mag)
fig.show()

In [ ]:
pylab.figure(figsize=(5,5))
freq, mag = compute_fft_plot_from_sample_rate(signal, sampling_rate=fs, center_frequency=0)
pylab.plot(freq, mag)
pylab.show()

In [ ]:
filtered_signal = filter_complex_signal(signal=signal, cutoff_frequency=75000, sample_rate=fs)

angles = np.angle(filtered_signal)
angle_diff = np.unwrap(np.diff(angles))

wavfile.write('/home/dominic/radio/data/raw_filter_before_diff.wav', int(fs), angle_diff)

In [ ]:
filtered_signal = filter_complex_signal(signal=signal, cutoff_frequency=75000, sample_rate=fs)

angles = np.angle(filtered_signal)
angle_diff = np.unwrap(np.diff(angles))

angle_diff_downsampled = angle_diff[::40]
downsampled_fs = fs / 40

wavfile.write('/home/dominic/radio/data/filter_before_diff_downsampled.wav', int(downsampled_fs), angle_diff_downsampled)

In [ ]:
filtered_signal = filter_complex_signal(signal=signal, cutoff_frequency=75000, sample_rate=fs)
filtered_signal_downsampled = filtered_signal[::10]
downsampled_fs = fs / 10

angles = np.angle(filtered_signal_downsampled)
angle_diff = np.unwrap(np.diff(angles))

freq, mag = compute_fft_plot_from_sample_rate(angle_diff, sampling_rate=downsampled_fs, center_frequency=0)

fig = go.Figure()
fig.add_scatter(x=freq, y=mag)
fig.show()

wavfile.write('/home/dominic/radio/data/filter_downsampled_before_diff.wav', int(downsampled_fs), angle_diff)

In [ ]:
filtered_signal = filter_complex_signal(signal=signal, cutoff_frequency=75000, sample_rate=fs)
filtered_signal_downsampled = filtered_signal[::10]
downsampled_fs = fs / 10

angles = np.angle(filtered_signal_downsampled)
angle_diff = np.unwrap(np.diff(angles))

angle_diff_downsampled = angle_diff[::5]
downsampled_fs = downsampled_fs / 5


freq, mag = compute_fft_plot_from_sample_rate(angle_diff_downsampled, sampling_rate=downsampled_fs, center_frequency=0)

fig = go.Figure()
fig.add_scatter(x=freq, y=mag)
fig.show()

wavfile.write('/home/dominic/radio/data/filter_before_diff_double_downsampled.wav', int(downsampled_fs), angle_diff_downsampled)

In [ ]:
filtered_signal = filter_complex_signal(signal=signal, cutoff_frequency=75000, sample_rate=fs)
filtered_signal_downsampled = filtered_signal[::10]
downsampled_fs = fs / 10

angles = np.angle(filtered_signal_downsampled)
angle_diff = np.diff(angles)

angle_diff_downsampled = angle_diff[::5]
downsampled_fs = downsampled_fs / 5


freq, mag = compute_fft_plot_from_sample_rate(angle_diff_downsampled, sampling_rate=downsampled_fs, center_frequency=0)

fig = go.Figure()
fig.add_scatter(x=freq, y=mag)
fig.show()

wavfile.write('/home/dominic/radio/data/filter_before_diff_double_downsampled_no_unwrap.wav', int(downsampled_fs), angle_diff_downsampled)

In [ ]:
filtered_signal = filter_complex_signal(signal=signal, cutoff_frequency=75000, sample_rate=fs)
filtered_signal_downsampled = filtered_signal[::10]
downsampled_fs = fs / 10

angles = np.angle(filtered_signal_downsampled)
angle_diff = np.diff(angles)

angle_diff[angle_diff < -np.pi] += 2*np.pi
angle_diff[angle_diff > np.pi] -= 2*np.pi


angle_diff_downsampled = angle_diff[::5]
downsampled_fs = downsampled_fs / 5


freq, mag = compute_fft_plot_from_sample_rate(angle_diff_downsampled, sampling_rate=downsampled_fs, center_frequency=0)

fig = go.Figure()
fig.add_scatter(x=freq, y=mag)
fig.show()

wavfile.write('/home/dominic/radio/data/filter_before_diff_double_downsampled_cust_unwrap.wav', int(downsampled_fs), angle_diff_downsampled)

In [ ]:
angles = np.angle(signal)
angle_diff = np.unwrap(np.diff(angles))

wavfile.write('/home/dominic/radio/data/no_filter_full_rate.wav', int(fs), angle_diff)

In [ ]:
angles = np.angle(signal)
angle_diff = np.unwrap(np.diff(angles))

filtered_signal = filter_real_signal(signal=angle_diff, cutoff_frequency=20000, sample_rate=fs)
filtered_signal_downsampled = filtered_signal[::40]
downsampled_fs = fs / 40


wavfile.write('/home/dominic/radio/data/filter_after_diff.wav', int(downsampled_fs), filtered_signal_downsampled)

In [ ]:
filtered_signal = filter_complex_signal(signal=signal, cutoff_frequency=75000, sample_rate=fs)

angles = np.angle(filtered_signal)
angle_diff = np.unwrap(np.diff(angles))

filtered_signal = filter_real_signal(signal=angle_diff, cutoff_frequency=20000, sample_rate=fs)
filtered_signal_downsampled = filtered_signal[::40]
downsampled_fs = fs / 40


wavfile.write('/home/dominic/radio/data/filter_both_diff.wav', int(downsampled_fs), filtered_signal_downsampled)

In [ ]:
filtered_signal = filter_complex_signal(signal=signal, cutoff_frequency=75000, sample_rate=fs)

angles = np.angle(filtered_signal)
angle_diff = np.diff(angles)

filtered_signal = filter_real_signal(signal=angle_diff, cutoff_frequency=20000, sample_rate=fs)
filtered_signal_downsampled = filtered_signal[::40]
downsampled_fs = fs / 40


wavfile.write('/home/dominic/radio/data/filter_both_diff_no_unwrap.wav', int(downsampled_fs), filtered_signal_downsampled)

In [ ]:
pylab.figure(figsize=(5,5))

pylab.psd(filtered_signal_downsampled, NFFT=1024, Fs=downsampled_fs, Fc=center_frequency)
pylab.xlabel('Frequency (MHz)')
pylab.ylabel('Relative power (dB)')

pylab.show()

In [ ]:
pylab.figure(figsize=(5,5))
freq, mag = compute_fft_plot_from_sample_rate(filtered_signal_downsampled, sampling_rate=downsampled_fs, center_frequency=0)
pylab.plot(freq, mag)
pylab.show()

In [ ]:
angles = np.angle(filtered_signal)
# Difference the phase angle and convert to +/-pi range.
angle_diff = np.unwrap(np.diff(angles))

In [ ]:
pylab.figure()

freq, mag = compute_fft_plot_from_sample_rate(angle_diff, sampling_rate=downsampled_fs, center_frequency=0)

pylab.plot(freq, mag)

pylab.show()

In [ ]:
filtered_angle_diff = filter_real_signal(signal=angle_diff, cutoff_frequency=20e3, sample_rate=fs)


# pylab.figure()

# freq, mag = compute_fft_plot_from_sample_rate(filtered_angle_diff, sampling_rate=fs, center_frequency=0)

# pylab.plot(freq, mag)

# pylab.show()

In [ ]:
angle_diff_downsampled = filtered_angle_diff[::25]
angle_diff_downsampled_fs = fs/25

# pylab.figure()

# freq, mag = compute_fft_plot_from_sample_rate(angle_diff_downsampled, sampling_rate=angle_diff_downsampled_fs, center_frequency=0)

# pylab.plot(freq, mag)

# pylab.show()

In [ ]:
wavfile.write('/home/dominic/radio/data/test.wav', int(downsampled_fs), filtered_angle_diff)

In [ ]:
pylab.figure()
pylab.plot(angle_diff_downsampled[:int(angle_diff_downsampled_fs*0.01)])
pylab.show()

In [ ]:
angle_diff_alt = np.diff(angles)

filtered_angle_diff_alt = filter_real_signal(signal=angle_diff_alt, cutoff_frequency=20e3, sample_rate=fs)
angle_diff_downsampled_alt = filtered_angle_diff_alt[::25]
angle_diff_downsampled_fs_alt = fs/25


In [ ]:
pylab.figure()
pylab.plot(angle_diff_downsampled_alt[:int(angle_diff_downsampled_fs_alt*0.01)])
pylab.show()

In [ ]:
wavfile.write('/home/dominic/radio/data/test_alt.wav', int(angle_diff_downsampled_fs_alt), angle_diff_downsampled_alt)

In [ ]:
test = np.diff(angles)
test_unwrap = np.unwrap(np.diff(angles))

test_diff = (test - test_unwrap) > 1e-4

In [ ]:
np.sum(test_diff) / float(len(test_diff))

In [ ]:
test_diff[1000:1010]

In [ ]:
test[1000:1010]

In [ ]:
test_unwrap[1000:1010]

In [ ]:
np.where(test_diff)

In [ ]:
test[63], test_unwrap[63]

In [ ]:
test = np.diff(angles)
test_unwrap = np.unwrap(test)



test_alt = test.copy()
test_alt[test_alt > 2*np.pi] -= 2*np.pi
test_alt[test_alt < -2*np.pi] += 2*np.pi

test_alt_diff = (test_alt - test_unwrap) > 1e-4
np.sum(test_alt_diff) / float(len(test_alt_diff))

In [ ]:
len(test) / 30, fs

In [ ]:
wavfile.write('/home/dominic/radio/data/test_raw.wav', int(fs), test)
wavfile.write('/home/dominic/radio/data/test_unwrapped.wav', int(fs), test_unwrap)
wavfile.write('/home/dominic/radio/data/test_mod2pi.wav', int(fs), test_alt)

In [ ]:
pylab.figure()
freq, mag = compute_fft_plot_from_sample_rate(test, sampling_rate=fs, center_frequency=0)
pylab.plot(freq, mag)
pylab.show()

In [ ]:
pylab.figure()
freq, mag = compute_fft_plot_from_sample_rate(test_unwrap, sampling_rate=fs, center_frequency=0)
pylab.plot(freq, mag)
pylab.show()

In [ ]:
pylab.figure()
freq, mag = compute_fft_plot_from_sample_rate(test_alt, sampling_rate=fs, center_frequency=0)
pylab.plot(freq, mag)
pylab.show()

In [ ]:
np.where(test_alt_diff)

In [ ]:
2*np.pi

In [ ]:
test_alt[63]

In [ ]:
test[63]

In [ ]:
test_unwrap[63]

In [ ]:
fs = 2048000
center_frequency = 98500000

signal = read_sdriq_data("/home/dominic/radio/data/long_test.sdriq")

audio_signal, audio_signal_fs = chunked_demodulate_signal(signal, sample_rate=fs)
# wavfile.write('/home/dominic/radio/data/test.wav', int(audio_signal_fs), audio_signal)

In [ ]:
np.min(audio_signal), np.max(audio_signal), np.mean(audio_signal)

In [ ]:
np.round(test_floats).astype(int)

In [ ]:
import numpy as np
import struct

In [ ]:
test_ints = np.random.randint(-65, 65, size=(100,))

test_ints = np.random.randint(-65, 65, size=(100,))


In [ ]:
format_string = ">b"  # big endian encoding of signed 8 bit integers
test_bytes = [struct.pack(format_string, i) for i in test_ints]

In [ ]:
test_bytes[0]

In [ ]:
0xC0

In [ ]:
2**7

In [ ]:
from scipy import signal
# from scipy.fft import fftshift
import matplotlib.pyplot as plt

In [ ]:
fs = 100
total_time = 10
time = np.linspace(0, total_time, fs*total_time)

In [ ]:
frequency = 10.0
values = np.cos(2*np.pi * frequency * time)

In [ ]:
f, t, Sxx = signal.spectrogram(values, fs=fs)

plt.pcolormesh(t, f, Sxx, shading='gouraud')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.show()

In [ ]:
values_sum = values + np.cos(2*np.pi * frequency*1.5 * time)

f, t, Sxx = signal.spectrogram(values_sum, fs=fs)

plt.pcolormesh(t, f, Sxx, shading='gouraud')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.show()

In [ ]:
values = np.cos(2*np.pi * frequency * time)
values_mult = values * np.cos(2*np.pi * 5 * time)


f, t, Sxx = signal.spectrogram(values_mult, fs=fs)

plt.pcolormesh(t, f, Sxx, shading='gouraud')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.show()

In [ ]:
values = np.cos(2*np.pi * frequency * time)
conv = np.power(np.e, 2j*np.pi*frequency*time)
values_mult = values * np.cos(2*np.pi * 5 * time)


f, t, Sxx = signal.spectrogram(values_mult, fs=fs)

plt.pcolormesh(t, f, Sxx, shading='gouraud')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.show()

In [ ]:
np.e

In [ ]:
np.power(np.e, np.pi * 1j)

In [ ]:
np.power(np.e, )

In [ ]:
values = np.cos(2*np.pi * frequency * time)


pylab.figure()

freq, mag = compute_fft_plot_from_sample_rate(values, sampling_rate=fs, center_frequency=0)

pylab.plot(freq, mag)

pylab.show()

In [ ]:
values = np.cos(2*np.pi * frequency * time)
conv = np.power(np.e, 2j*np.pi*frequency*time) # + np.power(np.e, -2j*np.pi*frequency*time)
values_mult = values * conv



pylab.figure()
freq, mag = compute_fft_plot_from_sample_rate(values_mult, sampling_rate=fs, center_frequency=0)
pylab.plot(np.fft.fftshift(freq), np.fft.fftshift(mag))
pylab.show()

In [ ]:
conv = np.power(np.e, -2j*np.pi*frequency*time)
values_mult_alt = values_mult * conv


pylab.figure()
freq, mag = compute_fft_plot_from_sample_rate(values_mult_alt, sampling_rate=fs, center_frequency=0)
pylab.plot(np.fft.fftshift(freq), np.fft.fftshift(mag))
pylab.show()

In [ ]:
np.abs(conv)

In [ ]:
pylab.figure()

freq, mag = compute_fft_plot_from_sample_rate(values, sampling_rate=fs, center_frequency=0)

pylab.plot(freq, mag)

pylab.show()

In [ ]:
from util.demodulation import chunked_demodulate_signal

In [ ]:
fs = 2048000
center_frequency = 98500000

signal = read_sdriq_data("/home/dominic/radio/data/long_test.sdriq")
len(signal) / fs

In [ ]:
audio, audio_fs = chunked_demodulate_signal(signal, fs)

In [ ]:
wavfile.write("data/test_long_decode_update.wav", int(audio_fs), audio)

In [ ]:
audio_fs

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fs = 1920E3
center_frequency = 446E6

signal = read_sdriq_data("/home/dominic/radio/data/ben_handheld_radio_test_center_446MHz_sample_1920KHz.raw")
signal = signal[int(fs*15):int(fs*25)]
len(signal) / fs

In [ ]:
audio, audio_fs = chunked_demodulate_signal(signal, fs)

In [ ]:
wavfile.write("data/ben_handheld_radio_test.wav", int(audio_fs), audio)

In [ ]:
plt.specgram(signal, Fs=fs)

In [ ]:
pylab.figure()
freq, mag = compute_fft_plot_from_sample_rate(signal, sampling_rate=fs, center_frequency=0)
pylab.plot(freq, mag)
pylab.show()

In [ ]:
filtered_signal = low_pass_filter_complex_signal(signal=signal, cutoff_frequency=25E3, sample_rate=fs)
filtered_fs = fs / 20
filtered_signal = filtered_signal[::20]

In [ ]:
filtered_fs

In [ ]:
pylab.figure()
freq, mag = compute_fft_plot_from_sample_rate(filtered_signal, sampling_rate=filtered_fs, center_frequency=0)
pylab.plot(freq, mag)
pylab.show()

In [ ]:
angle_diff = np.angle(
    np.conjugate(filtered_signal[:-1]) * filtered_signal[1:]
)


In [ ]:
pylab.figure()
freq, mag = compute_fft_plot_from_sample_rate(angle_diff, sampling_rate=filtered_fs, center_frequency=0)
pylab.plot(freq, mag)
pylab.show()

In [ ]:
wavfile.write("data/ben_handheld_radio_test.wav", int(filtered_fs), angle_diff)

In [ ]:
plt.psd(signal, Fs=fs)

In [ ]:
plt.psd(filtered_signal, Fs=filtered_fs)